In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import string
import numpy as np
import sklearn
# from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import HashingVectorizer
import math

In [ ]:
input_df = pd.read_csv('asset_dim.csv')
input_df['digital_asset'] = input_df['digital_asset'].str.lower()
input_df['cleaned_asset'] = input_df['digital_asset'].str.replace('[{}]'.format(string.punctuation), ' ')
input_df.dropna(subset=['cleaned_asset'], inplace=True)
input_df

# print(input_df.info(memory_usage='deep'))
# for col in input_df.columns:
  # input_df[col]=input_df[col].astype('category').cat.codes
# print(input_df.info(memory_usage='deep'))
input_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,asset_Sk,digital_asset,source_system_code,cleaned_asset
0,AD530941,afr-ug-mbarara28:77:44:b8:50:d5,CLRT,afr ug mbarara28 77 44 b8 50 d5
1,AD578674,eur-be-leuven00:1c:06:23:97:8e,CLRT,eur be leuven00 1c 06 23 97 8e
2,AD648865,naz-us-laa0:d3:7a:20:4f:e0,CLRT,naz us laa0 d3 7a 20 4f e0
3,AD685720,saz-br-camacariswitch,CLRT,saz br camacariswitch
4,AD705107,saz-br-juatuba10.140.12.22,CLRT,saz br juatuba10 140 12 22
...,...,...,...,...
276671,AD251561,irep login y token (mexico) - 2020,PT,irep login y token mexico 2020
276672,AD251305,bees us - 2022,PT,bees us 2022
276673,AD251299,bees digital coupon,PT,bees digital coupon
276674,AD251298,bees china - 2022,PT,bees china 2022


Fine Tuning

In [ ]:
import time
import datetime
import uuid
import glob

iter_times = []

# find matching records based on threshold
def find_matches(tuple):
  threshold = 0.89
  i = tuple[0] #this is the index
  col = 1 - tuple[1] #because we have received distance and not similarity
  np.put(col, i, 0) #to remove diagonal values as they represent similarity to itself
  loc = min(i, np.argmax(col, axis=0)) #show similarity to previously occuring item
  value = round(np.amax(col, axis=0), 2) #maximum similarity percentage
  if value >= threshold:
    return (loc, value) #return location
  else:
    return (i, value)

# similarity calculation
def similarity_calc(X):
  # similarities = sklearn.metrics.pairwise.pairwise_distances(X,X)
  similarities = sklearn.metrics.pairwise.cosine_distances(X,X)
  return similarities

# Create hash matrix of data
def hash_matrix_creator(chunked_df, col='cleaned_asset'):
  vectorizer = HashingVectorizer()
  X = vectorizer.fit_transform(chunked_df[col])
  return X

# both chunks of dataframe are appended to create holistic matrix
def chunk_n_append(df1, df2):
  chunked_df = df1.append(df2, ignore_index=False)
  return chunked_df

# final result preparation
def result_merge(chunked_df, similarities, chunk_size):
  matches = map(find_matches, enumerate(similarities))
  matches = pd.DataFrame(matches, columns=['Match_To', 'Match%'])
  matches = matches[chunk_size : chunk_size*2] # we need only second half as first half was for comparison purpose
  matches['row_number'] = np.arange(chunk_size, chunk_size+len(matches))
  matches = matches[matches['Match%'] >= 0.89].groupby("Match_To").filter(lambda x: len(x) > 1)
  # matches = matches.groupby("Match_To").filter(lambda x: len(x) > 1)
  chunked_df['row_number'] = np.arange(len(chunked_df))
  chunked_df = chunked_df[['row_number','asset_Sk']]
  matches = pd.merge(matches, chunked_df, on="row_number")
  matches = pd.merge(matches, chunked_df, left_on="Match_To", right_on="row_number", suffixes=("_matched","_matched_To"))
  matches = matches.drop(['Match_To', 'row_number_matched', 'row_number_matched_To'], axis=1)
  return matches

# Divide data into chunks for anlaysis
#adjust chunk size based on memory available
# save_output parameter is switched off for testing purposes
def chunked_analysis(df, chunk_size=5000, save_output=True):
  tic = time.perf_counter()
  session_id = str(uuid.uuid4())
  for i_start in tqdm(range(0, len(df), chunk_size), position=0, leave=False):
    i_end = i_start + chunk_size
    for j_start in tqdm(range(0, i_start+chunk_size, chunk_size), position=1, leave=False):
      j_end = j_start + chunk_size
      similarities = similarity_calc(hash_matrix_creator(chunk_n_append(df[j_start:j_end], df[i_start:i_end])))
      matches = result_merge(chunk_n_append(df[j_start:j_end], df[i_start:i_end]), similarities, chunk_size)
      
      if save_output == True:
        filename = str(uuid.uuid4())
        matches.to_parquet(f"{session_id}_{filename}.parquet")
  toc = time.perf_counter()
  iter_times.append(toc-tic)
  return toc-tic, session_id

input_df = input_df.sort_values(by="asset_Sk")
total_time, session_id = chunked_analysis(input_df, chunk_size=8000)

total_time = datetime.timedelta(seconds=sum(iter_times))
print(f"Total time taken {total_time} seconds")
# for i, value in enumerate(iter_times):
#   print(f"Iteration {i} taken {value:0.4f} seconds")

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# function to optimize chunk size
def optimize_chunk(df, alpha=.1):
  import plotly.express as px
  from IPython.display import clear_output

  max_len = len(df)
  chunk_size = 3000
  chunk_run_data = []
  while(chunk_size<=max_len):
    run_times = []
    for x in tqdm(range(0,3), leave=False): #increase iterations for accurate results
      run_time, session_id = chunked_analysis(df, chunk_size=chunk_size, save_output=False)
      run_times.append(run_time)
    print(f"Total Time {np.mean(run_times):0.4f} seconds for {chunk_size} records chunk")
    chunk_run_data.append((chunk_size, np.mean(run_times)))
    chunk_size = math.ceil(chunk_size + chunk_size * alpha)
    _df = pd.DataFrame(chunk_run_data, columns=['Chunk Size', 'Execution Time'])
    clear_output()
    fig = px.line(_df, x="Chunk Size", y="Execution Time", markers=True)
    fig.update_traces(textposition="bottom right")  
    fig.show()

# input_df = input_df.set_index(input_df['asset_Sk'])
input_df['row_number'] =np.arange(len(input_df))
optimize_chunk(input_df[:25000]) #chose sample data for quick benchmarking

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
filenames = glob.glob("*.xlsx")
finalexcelsheet = pd.DataFrame()
for file in filenames:
  df = pd.concat(pd.read_excel(
      file, sheet_name=None), ignore_index=True, sort=False)
  finalexcelsheet = finalexcelsheet.append(
      df, ignore_index=True)
finalexcelsheet.to_excel(r'Final_result.xlsx', index=False)

In [ ]:


df = pd.read_parquet('1a25802d-9279-429b-b58f-ec64a73dfdfd_0943550d-e54a-48cc-aa1c-afc3f13fef4f.parquet')
df.to_excel("result.xlsx", index=False, header=True)
